In [ ]:
import os
import sys
import shutil
import atexit
import logging

from ncclient import manager
from xml.etree import ElementTree
from ncclient.xml_ import to_ele
from ncclient.operations import RPCError

from flask import Flask, send_from_directory

app = Flask(__name__)
app.config['SECRET_KEY'] = 'secret'
action_namespace = 'http://example.com/ztp'
temporary_dir = 'tmp'
session = {}


In [ ]:
"""
Preparation: initial configuration
"""
def initial_config():

    # NSO credentials
    session['nso'] = {}
    session['nso']['host'] = "198.18.134.50"
    session['nso']['port'] = "2022"
    session['nso']['user'] = "admin"
    session['nso']['pass'] = "admin"

    # Read NSO RPC action for request of configuration file
    file_name = "config/config-action.xml"
    with open(file_name, 'r') as file:
        session['nso']['config-action'] = file.read()

    # Read NSO RPC action for request of onboarding
    file_name = "config/onboard-action.xml"
    with open(file_name, 'r') as file:
        session['nso']['onboard-action'] = file.read()

    # Create temporary directory that stores config files
    if not os.path.exists(temporary_dir):
        os.mkdir(temporary_dir)
        

In [ ]:
"""
Route /ztp/conf/v1/<serial> returns the contents of node <conf> as a reply to
calling an action over NETCONF.

The response is empty if:
 - the serial number does not exist
 - the NETCONF connection cannot be established
 - the RPC call fails
 - the action file is invalid
 - there is no <conf> node returned over NETCONF in the RPCReply

Throws an error if the RPC call fails.
"""
@app.route('/ztp/conf/v1/<serial>')
def conf_v1(serial):
    m = manager.connect(look_for_keys=False,
                        host=session['nso']['host'],
                        port=session['nso']['port'],
                        username=session['nso']['user'],
                        password=session['nso']['pass'],
                        hostkey_verify=False)
    try:
        # Call get config action
        if session['nso']['config-action'] is not None and len(session['nso']['config-action']) > 0:
            try:
                response = m.dispatch(to_ele(session['nso']['config-action'].format(SERIAL=serial)))
                configuration = response.data_ele.find(".//*{" + action_namespace + "}message")

                # Return a configuration file
                if configuration is not None:
                    with open(os.path.join(temporary_dir, serial), 'w') as file:
                        file.write(configuration.text)

                    return send_from_directory(temporary_dir, serial,
                                               as_attachment=True)
            except RPCError as e:
                print('Failed to make RPC request:', e)
    finally:
        m.close_session()

    return ''


In [ ]:
"""
Route /ztp/status/<serial>/<status> triggers the device onboarding
process in NSO if the its readiness status is 1.

Expected input parameters:
 - serial (serial number of the device)
 - status (readiness status of the device)

The response is empty all the time, even if:
 - the serial number does not exist
 - the NETCONF connection cannot be established
 - the RPC call fails
 - the action file for onboarding is invalid

Throws an error if the RPC call fails.
"""
@app.route('/ztp/status/<serial>/<int:status>', methods=['POST'])
def onboard(serial, status):

    m = manager.connect(look_for_keys=False,
                        host=session['nso']['host'],
                        port=session['nso']['port'],
                        username=session['nso']['user'],
                        password=session['nso']['pass'],
                        hostkey_verify=False)
    
    try:
        if serial and status == 1:
            # Call onboarding action
            if session['nso']['onboard-action'] is not None and len(session['nso']['onboard-action']) > 0:
                try:
                    m.dispatch(to_ele(session['nso']['onboard-action'].format(SERIAL=serial)))
                except RPCError as e:
                    print('Failed to make RPC request:', e)

    finally:
        m.close_session()

    return ''



In [7]:
if __name__ == '__main__':

    initial_config()
    # Expose server through all IP addresses, on port 8181
    app.run(host='0.0.0.0', port=8181)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:8181/ (Press CTRL+C to quit)
